# generate_error_table_sampled

## imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import os

%matplotlib inline

## load data

In [2]:
df = pd.read_csv('final_data/20230311_sampled.csv')
df

,benchmark,histogram,query,operator_count,operator_type,real_input,real_output,estimated_input,estimated_output,sampling_rate
0,TPC-DS,MaxDiffAreaHistogram,01,0,JoinHashLeft,2000000,558900,2000000.0,15290.688477,100
1,TPC-DS,MaxDiffAreaHistogram,01,1,JoinHashLeft,28795080,5580151,28795080.0,125474.312500,100
2,TPC-DS,MaxDiffAreaHistogram,01,2,TableScan,402,45,402.0,45.000000,100
3,TPC-DS,MaxDiffAreaHistogram,01,3,TableScan,65535,366,65535.0,329.850647,100
4,TPC-DS,MaxDiffAreaHistogram,03,0,TableScan,73049,6000,73049.0,6000.000000,100
...,...,...,...,...,...,...,...,...,...,...
4706,TPC-DS,MaxDiffAreaHistogram,99,1,JoinHashRight,30,28521632,30.0,2.638168,1
4707,TPC-DS,MaxDiffAreaHistogram,99,2,JoinHashRight,30,28539766,30.0,58.696945,1
4708,TPC-DS,MaxDiffAreaHistogram,99,3,JoinHashRight,20,28575799,20.0,1305.959473,1
4709,TPC-DS,MaxDiffAreaHistogram,99,4,JoinHashLeft,143997065,28647730,143997072.0,29056.955078,1


## generate errors

In [3]:
def generate_errors(df):
    df = df.copy()  # Convenience
    
    df['absolute_error'] = np.absolute(df['estimated_output'] - df['real_output'])
    df['relative_error'] = df['absolute_error'] / df['real_output']
    df['x'] = df['estimated_output'] / df['real_output']
    df['1/x'] = 1 / df['x']
    df['q_error'] = df[['x','1/x']].max(axis=1)
    df['real_output_lower_bound'] = df['real_output']
    df['real_output_lower_bound'] = df['real_output_lower_bound'].clip(1)
    df['estimated_output_lower_bound'] = df['estimated_output']
    df['estimated_output_lower_bound'] = df['estimated_output_lower_bound'].clip(1)
    df['pseudo_x'] = df['estimated_output_lower_bound'] / df['real_output_lower_bound']
    df['pseudo_1/x'] = 1 / df['pseudo_x']
    df['pseudo_q_error'] = df[['pseudo_x','pseudo_1/x']].max(axis=1)
    df['mean_squared_error'] = df['absolute_error'] ** 2
    df.drop(['x', '1/x', 'real_output_lower_bound', 'estimated_output_lower_bound', 'pseudo_x', 'pseudo_1/x'], axis=1, inplace = True)

    return df

## result

### with_NULLs

In [ ]:
df_with_errors = generate_errors(df)
df_with_errors

df_with_errors.groupby([#"operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

### without_NULLs

In [5]:
size_before = len(df)
df = df.drop(df[df.real_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'real_output' cell was zero.")
df = df.drop(df[df.estimated_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'estimated_output' cell was zero.")

df_with_errors = generate_errors(df)

results=[]

# print as the resulting table is cut by Jupyter
for operator in pd.unique(df_with_errors.operator_type):
    filtered = df_with_errors.query("operator_type == @operator")
    result = filtered.groupby(["operator_type",
                               "sampling_rate"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                                 mean_absolute_error=("absolute_error", np.mean),
                                                 mean_relative_error=("relative_error", np.mean),
                                                 mean_q_error=("q_error", np.mean),
                                                 mean_pseudo_q_error=("pseudo_q_error", np.mean))
    results.append(result)
    display(result)

Removed 0 (0.00%) rows because the 'real_output' cell was zero.
Removed 0 (0.00%) rows because the 'estimated_output' cell was zero.


root_mean_squared_error  mean_absolute_error  \
operator_type sampling_rate                                                 
JoinHashLeft  1                         2.949313e+07         9.975147e+06   
              2                         2.965874e+07         1.011303e+07   
              3                         3.336608e+07         1.112579e+07   
              4                         3.339577e+07         1.118673e+07   
              5                         3.313670e+07         1.103789e+07   
              6                         3.343526e+07         1.123677e+07   
              7                         3.342954e+07         1.129308e+07   
              8                         3.279237e+07         1.090917e+07   
              9                         3.294597e+07         1.104682e+07   
              10                        3.275026e+07         1.095097e+07   
              15                        3.237507e+07         1.089541e+07   
              20                        2.813583e+07         9.771600e+06   
              30                        5.954434e+07         1.313541e+07   
              40                        5.364697e+07         1.307452e+07   
              50                        4.602637e+07         1.223100e+07   
              100                       1.777888e+07         8.931268e+06   

                             mean_relative_error  mean_q_error  \
operator_type sampling_rate                                      
JoinHashLeft  1                        26.209566  3.399408e+10   
              2                        26.562192  3.713310e+10   
              3                        26.561689  2.227376e+10   
              4                        26.740590  2.069163e+10   
              5                        26.381731  1.952481e+10   
              6                        26.920738  1.831241e+10   
              7                        27.104921  1.752431e+10   
              8                        26.202243  1.509831e+10   
              9                        26.551910  1.484840e+10   
              10                       26.370675  1.332114e+10   
              15                       26.539142  8.253764e+09   
              20                       26.518565  7.702700e+09   
              30                       26.433429  3.786067e+09   
              40                       28.207430  2.479752e+09   
              50                       28.779955  1.862479e+09   
              100                      38.721741  9.758372e+08   

                             mean_pseudo_q_error  
operator_type sampling_rate                       
JoinHashLeft  1                    602708.562834  
              2                    618039.405499  
              3                    613330.075884  
              4                    612910.914912  
              5                    608611.262095  
              6                    619353.319426  
              7                    619962.044172  
              8                    460246.704808  
              9                    517727.845751  
              10                   472834.761591  
              15                   400114.841324  
              20                   352464.357905  
              30                   344456.586028  
              40                   357601.425993  
              50                   361155.653011  
              100                  549658.692800

root_mean_squared_error  mean_absolute_error  \
operator_type sampling_rate                                                 
TableScan     1                         3.490641e+06        906433.813381   
              2                         3.490586e+06        906419.551794   
              3                         3.390404e+06        861264.462388   
              4                         3.355956e+06        852514.417698   
              5                         3.303634e+06        834747.595767   
              6                         3.286893e+06        834971.549754   
              7                         3.252329e+06        826191.739270   
              8                         3.200812e+06        808768.408267   
              9                         3.101302e+06        767466.045940   
              10                        3.099407e+06        774945.738303   
              15                        2.884805e+06        710275.443203   
              20                        2.745553e+06        682918.944107   
              30                        2.408728e+06        599118.642561   
              40                        2.039332e+06        499530.021397   
              50                        1.707895e+06        418260.836397   
              100                       1.081389e+05         10798.913134   

                             mean_relative_error  mean_q_error  \
operator_type sampling_rate                                      
TableScan     1                         0.156188      4.058253   
              2                         0.156187      4.056022   
              3                         0.151091      3.242252   
              4                         0.150455      2.687934   
              5                         0.148222      2.340879   
              6                         0.149179      2.135166   
              7                         0.148541      1.977744   
              8                         0.146333      1.851382   
              9                         0.139757      1.729971   
              10                        0.142050      1.674137   
              15                        0.134750      1.447727   
              20                        0.132488      1.349934   
              30                        0.126394      1.245373   
              40                        0.116811      1.187262   
              50                        0.110901      1.156872   
              100                       0.079692      1.094076   

                             mean_pseudo_q_error  
operator_type sampling_rate                       
TableScan     1                         4.058253  
              2                         4.056022  
              3                         3.242252  
              4                         2.687934  
              5                         2.340879  
              6                         2.135166  
              7                         1.977744  
              8                         1.851382  
              9                         1.729971  
              10                        1.674137  
              15                        1.447727  
              20                        1.349934  
              30                        1.245373  
              40                        1.187262  
              50                        1.156872  
              100                       1.094076

root_mean_squared_error  mean_absolute_error  \
operator_type sampling_rate                                                 
JoinHashRight 1                         1.857953e+08         5.786804e+07   
              2                         1.797716e+08         5.531350e+07   
              3                         1.793241e+08         5.520091e+07   
              4                         1.735735e+08         5.219785e+07   
              5                         1.738700e+08         5.224444e+07   
              6                         1.589724e+08         4.546191e+07   
              7                         1.558680e+08         4.344910e+07   
              8                         1.732766e+08         5.205012e+07   
              9                         1.731435e+08         5.199880e+07   
              10                        1.713609e+08         5.160874e+07   
              15                        1.578886e+08         4.874108e+07   
              20                        1.454534e+08         4.514409e+07   
              30                        2.062429e+08         8.022111e+07   
              40                        1.526034e+08         5.831945e+07   
              50                        1.212026e+08         4.618133e+07   
              100                       1.821611e+07         1.135714e+07   

                             mean_relative_error  mean_q_error  \
operator_type sampling_rate                                      
JoinHashRight 1                         4.678403  2.316971e+06   
              2                         4.433104  1.812128e+06   
              3                         4.432982  8.021811e+05   
              4                         4.218321  1.111309e+06   
              5                         4.218171  7.953705e+05   
              6                         3.709871  3.558530e+05   
              7                         3.573942  1.865172e+05   
              8                         4.217625  1.769032e+05   
              9                         4.217434  1.140682e+05   
              10                        4.216422  7.757199e+04   
              15                        4.209687  1.540993e+04   
              20                        4.091606  5.000469e+03   
              30                        5.865965  5.573429e+02   
              40                        5.039252  2.497834e+02   
              50                        5.235455  1.226143e+02   
              100                       9.054123  5.974014e+07   

                             mean_pseudo_q_error  
operator_type sampling_rate                       
JoinHashRight 1                     1.810671e+06  
              2                     1.812128e+06  
              3                     8.021811e+05  
              4                     1.111309e+06  
              5                     7.953705e+05  
              6                     3.558530e+05  
              7                     1.865172e+05  
              8                     1.769032e+05  
              9                     1.140682e+05  
              10                    7.757199e+04  
              15                    1.540993e+04  
              20                    5.000469e+03  
              30                    5.573429e+02  
              40                    2.497834e+02  
              50                    1.226143e+02  
              100                   1.758855e+04

root_mean_squared_error  mean_absolute_error  \
operator_type sampling_rate                                                 
Aggregate     1                             203989.0             203989.0   
              2                             203989.0             203989.0   
              3                             203989.0             203989.0   
              4                             203989.0             203989.0   
              5                             203989.0             203989.0   
              6                             203989.0             203989.0   
              7                             203989.0             203989.0   
              8                             203989.0             203989.0   
              9                             203989.0             203989.0   
              10                            203989.0             203989.0   
              15                            203989.0             203989.0   
              20                            203989.0             203989.0   
              30                            203989.0             203989.0   
              40                            203989.0             203989.0   
              50                            203989.0             203989.0   
              100                           203989.0             203989.0   

                             mean_relative_error  mean_q_error  \
operator_type sampling_rate                                      
Aggregate     1                     18544.454545  18545.454545   
              2                     18544.454545  18545.454545   
              3                     18544.454545  18545.454545   
              4                     18544.454545  18545.454545   
              5                     18544.454545  18545.454545   
              6                     18544.454545  18545.454545   
              7                     18544.454545  18545.454545   
              8                     18544.454545  18545.454545   
              9                     18544.454545  18545.454545   
              10                    18544.454545  18545.454545   
              15                    18544.454545  18545.454545   
              20                    18544.454545  18545.454545   
              30                    18544.454545  18545.454545   
              40                    18544.454545  18545.454545   
              50                    18544.454545  18545.454545   
              100                   18544.454545  18545.454545   

                             mean_pseudo_q_error  
operator_type sampling_rate                       
Aggregate     1                     18545.454545  
              2                     18545.454545  
              3                     18545.454545  
              4                     18545.454545  
              5                     18545.454545  
              6                     18545.454545  
              7                     18545.454545  
              8                     18545.454545  
              9                     18545.454545  
              10                    18545.454545  
              15                    18545.454545  
              20                    18545.454545  
              30                    18545.454545  
              40                    18545.454545  
              50                    18545.454545  
              100                   18545.454545

In [ ]:
sns.set_context('paper')
sns.set(rc={'figure.figsize':(15,5)})
ax = sns.lineplot(x = 'THREAD_COUNT', y = 'BUILD_TIME', data = multi_bts.reset_index(),
            palette = 'Blues')
ax.set(xlabel='Thread Count', ylabel='Build Time in s')
plt.show()

In [6]:
results[0]

root_mean_squared_error  mean_absolute_error  \
operator_type sampling_rate                                                 
JoinHashLeft  1                         2.949313e+07         9.975147e+06   
              2                         2.965874e+07         1.011303e+07   
              3                         3.336608e+07         1.112579e+07   
              4                         3.339577e+07         1.118673e+07   
              5                         3.313670e+07         1.103789e+07   
              6                         3.343526e+07         1.123677e+07   
              7                         3.342954e+07         1.129308e+07   
              8                         3.279237e+07         1.090917e+07   
              9                         3.294597e+07         1.104682e+07   
              10                        3.275026e+07         1.095097e+07   
              15                        3.237507e+07         1.089541e+07   
              20                        2.813583e+07         9.771600e+06   
              30                        5.954434e+07         1.313541e+07   
              40                        5.364697e+07         1.307452e+07   
              50                        4.602637e+07         1.223100e+07   
              100                       1.777888e+07         8.931268e+06   

                             mean_relative_error  mean_q_error  \
operator_type sampling_rate                                      
JoinHashLeft  1                        26.209566  3.399408e+10   
              2                        26.562192  3.713310e+10   
              3                        26.561689  2.227376e+10   
              4                        26.740590  2.069163e+10   
              5                        26.381731  1.952481e+10   
              6                        26.920738  1.831241e+10   
              7                        27.104921  1.752431e+10   
              8                        26.202243  1.509831e+10   
              9                        26.551910  1.484840e+10   
              10                       26.370675  1.332114e+10   
              15                       26.539142  8.253764e+09   
              20                       26.518565  7.702700e+09   
              30                       26.433429  3.786067e+09   
              40                       28.207430  2.479752e+09   
              50                       28.779955  1.862479e+09   
              100                      38.721741  9.758372e+08   

                             mean_pseudo_q_error  
operator_type sampling_rate                       
JoinHashLeft  1                    602708.562834  
              2                    618039.405499  
              3                    613330.075884  
              4                    612910.914912  
              5                    608611.262095  
              6                    619353.319426  
              7                    619962.044172  
              8                    460246.704808  
              9                    517727.845751  
              10                   472834.761591  
              15                   400114.841324  
              20                   352464.357905  
              30                   344456.586028  
              40                   357601.425993  
              50                   361155.653011  
              100                  549658.692800

In [4]:
size_before = len(df)
df = df.drop(df[df.real_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'real_output' cell was zero.")
df = df.drop(df[df.estimated_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'estimated_output' cell was zero.")

df_with_errors = generate_errors(df)

# print as the resulting table is cut by Jupyter
for operator in pd.unique(df_with_errors.operator_type):
    filtered = df_with_errors.query("operator_type == @operator")
    result = filtered.groupby(["operator_type",
                               "sampling_rate"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                                 mean_absolute_error=("absolute_error", np.mean),
                                                 mean_relative_error=("relative_error", np.mean),
                                                 mean_q_error=("q_error", np.mean),
                                                 mean_pseudo_q_error=("pseudo_q_error", np.mean))
    display(result)
    melted_results = pd.melt(result.reset_index(),
                             value_vars=["root_mean_squared_error", "mean_absolute_error",
                                         "mean_relative_error", "mean_q_error", "mean_pseudo_q_error"],
                             var_name="metric")
    g = sns.FacetGrid(result, col="sampling_rate", row="metric", sharey=False, aspect=2)
    g.map_dataframe(sns.barplot, x="sampling_rate", y="value", hue="sampling_rate", palette="Set3")
    g.add_legend()
    plt.savefig(f"{operator}_facet.pdf")
    plt.show()
    display(g)


Removed 0 (0.00%) rows because the 'real_output' cell was zero.
Removed -411 (8.72%) rows because the 'estimated_output' cell was zero.


root_mean_squared_error  mean_absolute_error  \
operator_type sampling_rate                                                 
JoinHashLeft  1                         2.949313e+07         9.975147e+06   
              2                         2.965874e+07         1.011303e+07   
              3                         3.336608e+07         1.112579e+07   
              4                         3.339577e+07         1.118673e+07   
              5                         3.313670e+07         1.103789e+07   
              6                         3.343526e+07         1.123677e+07   
              7                         3.342954e+07         1.129308e+07   
              8                         3.279237e+07         1.090917e+07   
              9                         3.294597e+07         1.104682e+07   
              10                        3.275026e+07         1.095097e+07   
              15                        3.237507e+07         1.089541e+07   
              20                        2.813583e+07         9.771600e+06   
              30                        5.954434e+07         1.313541e+07   
              40                        5.364697e+07         1.307452e+07   
              50                        4.602637e+07         1.223100e+07   
              100                       1.777888e+07         8.931268e+06   

                             mean_relative_error  mean_q_error  \
operator_type sampling_rate                                      
JoinHashLeft  1                        26.209566  3.399408e+10   
              2                        26.562192  3.713310e+10   
              3                        26.561689  2.227376e+10   
              4                        26.740590  2.069163e+10   
              5                        26.381731  1.952481e+10   
              6                        26.920738  1.831241e+10   
              7                        27.104921  1.752431e+10   
              8                        26.202243  1.509831e+10   
              9                        26.551910  1.484840e+10   
              10                       26.370675  1.332114e+10   
              15                       26.539142  8.253764e+09   
              20                       26.518565  7.702700e+09   
              30                       26.433429  3.786067e+09   
              40                       28.207430  2.479752e+09   
              50                       28.779955  1.862479e+09   
              100                      38.721741  9.758372e+08   

                             mean_pseudo_q_error  
operator_type sampling_rate                       
JoinHashLeft  1                    602708.562834  
              2                    618039.405499  
              3                    613330.075884  
              4                    612910.914912  
              5                    608611.262095  
              6                    619353.319426  
              7                    619962.044172  
              8                    460246.704808  
              9                    517727.845751  
              10                   472834.761591  
              15                   400114.841324  
              20                   352464.357905  
              30                   344456.586028  
              40                   357601.425993  
              50                   361155.653011  
              100                  549658.692800

KeyError: 'metric'